In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time

In [2]:
with open('negeri.json') as fopen:
    negeris = json.load(fopen)

max_page = 100
processed = ['']
negeris = [r['slugId'] for r in negeris['data']['places']['popular']['items'] if r['slugId'] not in processed]
negeris

['selangor',
 'johor',
 'kuala-lumpur',
 'negeri-sembilan',
 'penang',
 'pahang',
 'kelantan',
 'kedah',
 'perak',
 'labuan',
 'melaka',
 'terengganu',
 'perlis',
 'putrajaya',
 'sabah',
 'sarawak']

In [3]:
headers = """
accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
accept-encoding: gzip, deflate, br
accept-language: en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7
cache-control: max-age=0
cookie: KP_UID=35430865-8c0a-d297-d836-4abd6c36ca88; feature-RWD-6082=false; _gcl_au=1.1.1750258107.1591344254; _ga=GA1.3.1576862493.1591344255; _gid=GA1.3.820041376.1591344255; gdpr_notification=yes; scarab.visitor=%226DBDAEC813BAD72D%22; ASP.NET_SessionId=o2dtvxcqrim4y0fxg4byty0j; anonymoususerid=47733004-62ab-4c77-bf4d-804f23e527d9; scarab.mayAdd=%5B%7B%22i%22%3A%228775647%22%7D%5D; scarab.profile=%228775647%7C1591344450%22; KP_UIDz=3u1wq3j%2BxrKi5q0goXvjoA%3D%3D%3A%3AZJJj906tLlDktNPhXOkOKF2RVcWoavII5OaFeNm%2Fd4J0%2FURLl2EdOM6xzlOazuNcm8LbGdyXVxBw5xsVq3%2BWVeX6mriRnDkTaIoAS%2BLYefSpqE%2BSWVx4qCmmRyOLQbmvb9u0O134eI%2BoRo5IFSpZSAPJ1GA3tmnQJKYn3%2BGQvtCdxDAiwQL9FJcrHX5wZ6riv8icKx3HOcdJXCAtsni9whlOQ9BlIFl6uYO%2BJ3yS6I74lbbWgGpEWaVlKPaXXBQ7MCPEdGCi6zAiwW4Aa4Ud6sk0U1VYC4qtGwxHny5prjsPgmgnzH8bIbM97w%2FXA1zFtVxOLN0XuNmhZQDV7xKhLbFwasTW5Z4Xht6UZRwxMkX5z7x5A3EfT2SnWrEhOVUKQkb7dlpCMig3HmtZ1%2BBLmz4wJMolVSdWIog23R1g6DOtVRKsi0eM%2BJJo%2FADZuB4uKIuXX4NhtSQvYpfmkBqaGl29kNiwkNoPVX6sCpwP4D3DgfwrUFY0Yas%2FQydtg5bUOWccWuYnHh2v4T9EpoZlK6Bbv%2BC3IotlB9ztlL%2F091D%2F5%2FPh5soWv0ojRXmUi0vb
sec-fetch-dest: document
sec-fetch-mode: navigate
sec-fetch-site: same-origin
sec-fetch-user: ?1
upgrade-insecure-requests: 1
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36
"""

In [4]:
combined = {}

for row in headers.split('\n')[1:-1]:
    splitted = row.split(':')
    key = splitted[0]
    val = ''.join(splitted[1:])
    combined[key] = val.strip()
    
combined

{'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
 'cache-control': 'max-age=0',
 'cookie': 'KP_UID=35430865-8c0a-d297-d836-4abd6c36ca88; feature-RWD-6082=false; _gcl_au=1.1.1750258107.1591344254; _ga=GA1.3.1576862493.1591344255; _gid=GA1.3.820041376.1591344255; gdpr_notification=yes; scarab.visitor=%226DBDAEC813BAD72D%22; ASP.NET_SessionId=o2dtvxcqrim4y0fxg4byty0j; anonymoususerid=47733004-62ab-4c77-bf4d-804f23e527d9; scarab.mayAdd=%5B%7B%22i%22%3A%228775647%22%7D%5D; scarab.profile=%228775647%7C1591344450%22; KP_UIDz=3u1wq3j%2BxrKi5q0goXvjoA%3D%3D%3A%3AZJJj906tLlDktNPhXOkOKF2RVcWoavII5OaFeNm%2Fd4J0%2FURLl2EdOM6xzlOazuNcm8LbGdyXVxBw5xsVq3%2BWVeX6mriRnDkTaIoAS%2BLYefSpqE%2BSWVx4qCmmRyOLQbmvb9u0O134eI%2BoRo5IFSpZSAPJ1GA3tmnQJKYn3%2BGQvtCdxDAiwQL9FJcrHX5wZ6riv8icKx3HOcdJXCAtsni9whlOQ9BlIFl6uYO%2BJ3yS6I

In [5]:
from tqdm import tqdm

for negeri in negeris:
    print(negeri)
    
    for i in tqdm(range(max_page)):
        try:
            url = f'https://www.iproperty.com.my/rent/{negeri}/all-residential/?l1&page={i + 1}'
            r = requests.get(url,headers = combined)
            bs = BeautifulSoup(r.content)
            scripts = bs.findAll('script')
            r = [script.get_text() for script in scripts if 'window.__INITIAL_STATE__' in script.get_text()][0]
            r = re.sub(r'[ ]+', ' ', r).strip()
            r = r.strip().replace('window.__INITIAL_STATE__ = ', '').replace(';\n window.__RENDER_APP_ERROR__ = false;\n window.__SERVICE_ENV__ = "production";\n window.__PUBLIC_PATH__= "https://assets-cdn.iproperty.com.my/assets/";', '')
            with open(f'{negeri}-page-{i + 1}.json', 'w') as fopen:
                json.dump(json.loads(r), fopen)
            time.sleep(1)
        except Exception as e:
            print(i)
            print(e)

  0%|          | 0/100 [00:00<?, ?it/s]

selangor


  0%|          | 0/100 [00:00<?, ?it/s]

johor


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

sabah


  0%|          | 0/100 [00:00<?, ?it/s]

sarawak


100%|██████████| 100/100 [02:52<00:00,  1.73s/it]
